<a href="https://colab.research.google.com/github/nedlecky/Colabs/blob/main/SensorFusionDataAnalyzer0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SensorFusion Data Recording Analyzer 0.1
---
<br>Version 0.1
<br>12/14/2023
<br>Ned Lecky ned@lecky.com
<br>
<br>This demo code ingests TI data recordings from SensorFusion and generates output analysis files. This is intended as an initial example of what we can do!

In [35]:
# Standard Imports
import os
import pandas as pd
import datetime
import matplotlib.pyplot as plt


In [36]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
class SFD:
  '''
  Simple class to import, analyze, and graph SensorFusion reports.
  '''
  path = 'PLEASE SET ME'

  def __init__(self, filename):
    self.read_file(filename)

  def remove_file(self, extensions):
    for ext in extensions:
      try:
        os.remove(f'{SFD.path}{self.filename_noext}{ext}')
      except:
        pass

  def read_file(self, filename):
    print(f'SFD.read_file({filename})')
    self.filename = os.path.basename(filename)
    self.filename_noext = self.filename[0:-4]
    self.df = pd.read_csv(f'{SFD.path}{filename}', index_col=0, parse_dates=True)

    # Remove any files that were (probably?) generated for this CSV
    self.remove_file(['.txt', '.jpg', '_1.jpg', '_2.jpg', 'df.jpg'])

  def show_stats(self):
    filename = f'{SFD.path}{self.filename_noext}.txt'
    with open(filename, 'w') as file:
      file.write(f'SensorFusionData Statistics\n')
      file.write(f'   Filename: {self.filename}\n')
      file.write(f'  Generated: {datetime.datetime.now()}\n\n')
      file.write(f'{self.df.head()}\n...\n\n')
      file.write(f'{self.df.describe()}\n')

    with open(filename, 'r') as file:
      return file.read()

  def plot1(self):
    plt.ioff()
    ax = self.df.plot(
        title=self.filename, figsize=(20,10), layout=(4,1),
        subplots=[('Range',), ('Strength',), ('Elevation', 'Azimuth'), ('Velocity',)])
    #plt.show()
    ax[0,0].get_figure().savefig(f'{SFD.path}{self.filename_noext}_1.jpg')

  def plot2(self):
    plt.ioff()
    fig = plt.figure(figsize=(20,10))

    ax = fig.add_subplot(411)
    ax.plot(self.df.index, self.df["Range"], color='blue')

    ax = fig.add_subplot(412)
    ax.plot(self.df.index, self.df["Strength"], color='purple')

    ax = fig.add_subplot(413)
    ax.plot(self.df.index, self.df["Azimuth"], color='orange')
    ax.plot(self.df.index, self.df["Elevation"], color='green')

    ax = fig.add_subplot(414)
    ax.plot(self.df.index, self.df["Velocity"], color='red')

    #plt.show()
    fig.savefig(f'{SFD.path}{self.filename_noext}_2.jpg')


In [41]:
# Get the list of all files and directories
SFD.path = "drive/MyDrive/SensorFusionData/"
file_listing = os.listdir(SFD.path)

file_listing.sort()
for filename in file_listing:
  if filename.endswith('.csv'):
      sf = SFD(filename)
      print(sf.show_stats())
      sf.plot1()
      #sf.plot2()


SFD.read_file(recording-TI2-2023-12-16T08-00-41.csv)
SensorFusionData Statistics
   Filename: recording-TI2-2023-12-16T08-00-41.csv
  Generated: 2023-12-17 13:14:49.227223

                        Range  Elevation  Azimuth  Velocity  Strength
Datetime                                                             
2023-12-17 00:41:12  6.891750      -0.36    -0.01   0.00000      6.00
2023-12-17 00:41:12  6.575500       0.28     0.07   0.13916     15.00
2023-12-17 00:41:12  6.512250       0.29     0.10   0.13916     16.00
2023-12-17 00:41:18  7.144500      -0.46    -0.26   0.13916      6.00
2023-12-17 00:41:18  7.081501      -0.45    -0.30   0.13916      5.76
...

            Range   Elevation     Azimuth    Velocity    Strength
count  799.000000  799.000000  799.000000  799.000000  799.000000
mean     5.875130    0.184606    0.047860    0.960474   15.899975
std      1.706237    0.297348    0.109685    1.381346    9.713609
min      1.454250   -0.910000   -0.390000    0.000000    5.000000
25